In [2]:
!git clone https://github.com/mateoKutnjak/pose_estimation.git

Cloning into 'pose_estimation'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 88 (delta 34), reused 74 (delta 23), pack-reused 0
Unpacking objects: 100% (88/88), done.


In [0]:
import sys
sys.path.append('/content/pose_estimation')
# sys.path.append('/content/pose_estimation')

In [4]:
from google.colab import drive

import os
import pprint

import models
import dataset
import tensorflow as tf
import keras.backend as k
import argparse

/content/pose_estimation/dataset.py:32: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(input_shape), 3, "Input must have 3 dimensions")
/content/pose_estimation/dataset.py:33: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (len(output_shape), 3, "Output must have 3 dimensions")
/content/pose_estimation/dataset.py:34: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (input_shape[-1], 3, "Input channels dimension must be three (RGB image)")
/content/pose_estimation/dataset.py:35: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert (output_shape[-1], self.joints_num, "Output channels dimension must be same as joints number")
Using TensorFlow backend.


In [5]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
if 'COLAB_TPU_ADDR' not in os.environ:
    print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
    tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
    print ('TPU address is', tpu_address)

    with tf.Session(tpu_address) as session:
        devices = session.list_devices()
    
    print('TPU devices:')
    pprint.pprint(devices)

('TPU address is', 'grpc://10.101.139.162:8470')
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 11914972315094709071),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6306679860051323200),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8502460612377582279),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14205959448577762562),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 861414463870873758),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13953024755326988358),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 9441595164094383576),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 9756303823549310632),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 97423

In [0]:
IMAGES_DIR = '/content/drive/My Drive/_images'
ANNOTATIONS_JSON_FILE = '/content/drive/My Drive/_annotations/annotations.json'
INPUT_SHAPE = (256, 256, 3)
OUTPUT_SHAPE = (64, 64, 16)
BATCH_SIZE = 8
STACKS_NUM = 4

In [0]:
mpii_dataset = dataset.MPII_dataset(
        images_dir=IMAGES_DIR,
        annots_json_filename=ANNOTATIONS_JSON_FILE,
        input_shape=INPUT_SHAPE,
        output_shape=OUTPUT_SHAPE,
        type='train'
    )

In [0]:
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 1.0

# k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
hg_model = models.HourglassModel(
    dataset=mpii_dataset,
    batch_size=BATCH_SIZE,
    stacks_num=STACKS_NUM,
)

In [0]:
hg_model.build()

In [23]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

hg_model.model = tf.contrib.tpu.keras_to_tpu_model(
    hg_model.model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

INFO:tensorflow:Querying Tensorflow master (grpc://10.101.139.162:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 11914972315094709071)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 6306679860051323200)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8502460612377582279)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 14205959448577762562)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 861414463870873758)
INFO:tensorflow:*** Available Device: _DeviceAttributes

In [24]:
hg_model.train(epochs=5)

Epoch 1/5
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(1,), dtype=tf.int32, name=u'core_id_10'), TensorSpec(shape=(1, 256, 256, 3), dtype=tf.float32, name=u'input_3_10'), TensorSpec(shape=(1, 64, 64, 16), dtype=tf.float32, name=u'0_head_conv_1x1_third-heatmap_target_80'), TensorSpec(shape=(1, 64, 64, 16), dtype=tf.float32, name=u'1_head_conv_1x1_third-heatmap_target_80'), TensorSpec(shape=(1, 64, 64, 16), dtype=tf.float32, name=u'2_head_conv_1x1_third-heatmap_target_80'), TensorSpec(shape=(1, 64, 64, 16), dtype=tf.float32, name=u'3_head_conv_1x1_third-heatmap_target_80')]
INFO:tensorflow:Overriding default placeholder.


ValueError: ignored